In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#Adapted from: 
# lqr_neural_error_dynamics.ipynb

import scipy.io as sio
import scipy.stats as sio_stat
import scipy.interpolate
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 

import os
import pickle
import sys
import copy

import time
import pylab as pl
from IPython import display

from bmi_dynamics_code import behavior_co_obs as bmi_b
from bmi_dynamics_code import data_for_v as preeyacode
from bmi_dynamics_code import util as bmi_util
from bmi_dynamics_code import analyze_lqr

import timeit
from sklearn import decomposition as skl_decomp

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

%matplotlib notebook

In [11]:
#Load a base file for simulation parameters: 
ext_list =['.png', '.eps']

plot_dir = r'D:\Dropbox\BMI_co_obs_paper\data\vivek\lqr'

home_dir = r'D:\Dropbox\Code\preeya_co_obstacle'
data_dir = r'D:\Dropbox\Data\preeya_co_obstacle'
data_path = os.path.join(data_dir, 'for_v_df.pkl')
print(os.path.exists(data_path))
d = pickle.load(open(data_path))

#UNPACK: 
decoder = d['decoder']
pre = d['pre']
num_targets = d['num_targets']
sample_period = d['sample_period']
target_hold_time = d['target_hold_time']
num_neurons = d['num_neurons']
target_color = d['target_color']
obs_big = d['obs_big']
obs_small = d['obs_small']
target_r_task = d['target_r_task'] #target_r_task = {0:1.7, 1:2.0}
cursor_r = d['cursor_r']
target_r_eff = d['target_r_eff']
target_pos = d['target_pos']
target_angle = np.arctan2(target_pos[:,1], target_pos[:,0])
target_dist = np.linalg.norm(target_pos, axis=1)
obs_pos = d['obs_pos']

tc_meta = d['tc_meta']
task_list = d['task_list']
num_tasks = len(task_list)
task2idx = d['task2idx']
idx2task = bmi_util.invert_dic(task2idx)
task2tc = d['task2tc']
tc_list = d['tc_list']
df = d['df']
df_label = d['df_label']

task_color = ['k', 'r']

#Pre-processing: 

bmi_b.preprocess_bmi_df(df, target_pos, pre, num_tasks, num_targets)
num_cw = 2
cw_label = ['cw', 'ccw']
task_rot_list = [0,1.1,1.2]

#Binning: 
#MAGNITUDE:
num_mag_bins = 5
num_mag_bins_analyze = 4
# mag_bin_perc = np.array([0,25,50,75,100])
mag_bin_perc = np.concatenate([np.linspace(0,95,5),np.array([100])])
#ANGLE:
num_angle_bins = 8
T0_angle = -3*(2*np.pi)/8
mag_bin, mag_bin_edges, mag_bin_c, angle_bin_c, angle_bin, angle_bin_edges =\
bmi_b.def_command_bin(df, mag_bin_perc, num_angle_bins, T0_angle)
#CENTER ANGLE DATA FOR BINNING (based on bin boundaries): 
angle_center_for_binning = bmi_b.df_center_angle_for_binning(df, angle_bin)
#BIN: 
bmi_b.df_bin_command(df, mag_bin_edges,angle_bin_edges)

#Pre-processing continued:

#Center angles to command bin, target: 
bmi_b.center_df_angle(df, angle_bin_c, target_angle)

True
('angle_center:', 22.5)
('min centered angle:', -157.47162455426732)
('max centered angle:', 202.46708261619673)


In [4]:
a_list = ['grom', 'jeev']
a2num_days = {'grom':9, 'jeev':4}

In [5]:
#Dynamics models: 
dm_list = ['full', 'decoder_null'] #dynamics, decoder_null dynamics
num_trials = 20 #number of trials simulated with lqr
# dm_files = {dm_list[0]:'lqr_sim_'+str(num_trials)+'trials.pkl',\
#             dm_list[1]:'lqr_sim_'+str(num_trials)+'trials_decoder_null_dyn.pkl'}

dm_files = {dm_list[0]:'lqr_sim_Q2_full.pkl',\
            dm_list[1]:'lqr_sim_Q2_decoder_null.pkl'}


In [6]:
model_list_analyze = \
[('full', 'n_do'), \
 ('decoder_null', 'n_do'), \
 ('full', 'n_o'), \
 ('decoder_null', 'n_o')]

mla = model_list_analyze

In [8]:
#Information for analysis: 
save_home = r'D:\DATA\bmi_co_obs_backup'

# #analyze neural diff: 
# p_sig_match = 0.4 #0.05
# shuffle_bool = True
# num_shuffle = 10
# min_obs = 15

# #Save information
# save_file = \
# 'analysis_lqr_sim_Q2_full_vs_decoder_null_'+str(num_trials)+'trials_'+ str(num_shuffle) + 'shuffles' +'.pkl'
# print(save_file)

In [115]:
debug_bool = False

for a in [a_list[1]]: # [a_list[0]]:
    for d in [0]: #range(a2num_days[a]):
        t_iter_start = timeit.default_timer()
        print(a,d)
        load_dir = os.path.join(save_home, a, str(d))
        
        if not debug_bool:
            #lqr sim paths for dynamics models (dm):
            sim_dm_paths = {}
            for dm in dm_list:
                sim_dm_paths[dm] = os.path.join(load_dir, dm_files[dm])
            lqr_dm, df_pool = analyze_lqr.pool_lqr_sim_across_dm(dm_list, sim_dm_paths)

        #Load some params, must be the same across dynamics models: 
        target_list = lqr_dm[dm_list[0]]['target_list'] 
        task_rot_list = lqr_dm[dm_list[0]]['task_rot_list'] 
        num_neurons = lqr_dm[dm_list[0]]['num_neurons']
        #Decoder:
        Kn = lqr_dm[dm_list[0]]['Kn']
        #decoder space
        u,s,vh = np.linalg.svd(Kn.T)
        u_d = u[:,:2]        
        
        #labels for movements, commands, neurons:
        move_list, c_list, n_list = \
        analyze_lqr.def_var_lists(num_targets, task_rot_list, num_mag_bins_analyze, num_angle_bins, num_neurons)

('jeev', 0)


In [116]:
move_list

[(0, 0),
 (0, 1.1),
 (0, 1.2),
 (1, 0),
 (1, 1.1),
 (1, 1.2),
 (2, 0),
 (2, 1.1),
 (2, 1.2),
 (3, 0),
 (3, 1.1),
 (3, 1.2),
 (4, 0),
 (4, 1.1),
 (4, 1.2),
 (5, 0),
 (5, 1.1),
 (5, 1.2),
 (6, 0),
 (6, 1.1),
 (6, 1.2),
 (7, 0),
 (7, 1.1),
 (7, 1.2)]

In [117]:
lqr_dm['decoder_null']['move_lqr_n'].keys()

[(4, 1.1, 'n_d'),
 (3, 0, 'n_null'),
 (3, 1.2, 'n_do'),
 (0, 1.1, 'n_null'),
 (5, 1.2, 'n_d'),
 (4, 1.2, 'n_null'),
 (2, 1.2, 'n_null'),
 (1, 1.2, 'n_null'),
 (0, 1.2, 'n_d'),
 (6, 0, 'n_o'),
 (3, 1.2, 'n_o'),
 (2, 0, 'n_null'),
 (3, 0, 'n_d'),
 (6, 1.2, 'n_null'),
 (5, 1.2, 'n_null'),
 (4, 1.2, 'n_d'),
 (7, 0, 'n_do'),
 (0, 0, 'n_do'),
 (2, 1.2, 'n_d'),
 (1, 0, 'n_do'),
 (1, 1.2, 'n_d'),
 (3, 1.1, 'n_do'),
 (7, 1.2, 'n_o'),
 (1, 1.1, 'n_do'),
 (7, 1.1, 'n_o'),
 (5, 0, 'n_null'),
 (1, 0, 'n_o'),
 (5, 1.1, 'n_o'),
 (6, 1.2, 'n_d'),
 (7, 1.1, 'n_do'),
 (5, 1.1, 'n_do'),
 (6, 1.1, 'n_d'),
 (4, 0, 'n_o'),
 (2, 1.1, 'n_null'),
 (5, 0, 'n_d'),
 (7, 1.2, 'n_do'),
 (1, 1.1, 'n_o'),
 (0, 0, 'n_null'),
 (3, 1.1, 'n_null'),
 (6, 1.1, 'n_null'),
 (4, 1.1, 'n_o'),
 (0, 1.2, 'n_o'),
 (2, 1.1, 'n_d'),
 (3, 1.2, 'n_d'),
 (2, 0, 'n_do'),
 (1, 1.2, 'n_do'),
 (3, 0, 'n_do'),
 (6, 0, 'n_d'),
 (7, 0, 'n_o'),
 (0, 1.2, 'n_do'),
 (0, 0, 'n_d'),
 (4, 1.2, 'n_o'),
 (0, 1.1, 'n_do'),
 (3, 1.1, 'n_d'),
 (0, 1.1,

In [120]:
K.shape

(20L, 25L)

In [121]:
Kn.shape

(5L, 20L)

In [125]:
for md in [('decoder_null', 'n_do'), ('decoder_null', 'n_o'), ('full', 'n_do')]:
    print('')
    print(md)
    m = move_list[-3] #movement condition
    print(m)
    move_lqr = lqr_dm[md[0]]['move_lqr_n'][m[0], m[1], md[1]]
    
    num_seg = len(move_lqr['K_list'])
    
    for seg in range(num_seg):
        print('seg', seg)
        K_list = move_lqr['K_list'][seg]
        num_samples = K_list.shape[1]-1
        print(num_samples)
        for i in range(num_samples):
            #print(i)
            K = K_list[0,i]
            u,s,vh = np.linalg.svd(K)
            u_dyn = u[:,:2]
            angles = scipy.linalg.subspace_angles(u_dyn, u_d)

            print(i, np.rad2deg(angles))
    


('decoder_null', 'n_do')
(7, 0)
('seg', 0)
38
(0, array([3.78942926e-14, 2.42221957e-14]))
(1, array([2.75232815e-14, 1.96681029e-14]))
(2, array([3.82979685e-14, 2.15051515e-14]))
(3, array([2.67694948e-14, 2.44301963e-14]))
(4, array([4.41157173e-14, 1.17439575e-14]))
(5, array([5.02935707e-14, 2.81467945e-14]))
(6, array([4.14523723e-14, 1.82389032e-14]))
(7, array([3.71357489e-14, 1.46002548e-14]))
(8, array([2.77183209e-14, 2.26133298e-14]))
(9, array([3.76235483e-14, 2.20253894e-14]))
(10, array([2.63836401e-14, 1.91037262e-14]))
(11, array([3.24024211e-14, 1.35114400e-14]))
(12, array([3.28789967e-14, 1.40997692e-14]))
(13, array([3.03980518e-14, 2.21505045e-14]))
(14, array([4.07343180e-14, 2.26394163e-14]))
(15, array([3.54943829e-14, 1.51527652e-14]))
(16, array([4.03388769e-14, 3.39389879e-14]))
(17, array([3.30815232e-14, 2.27906696e-14]))
(18, array([4.09849133e-14, 2.50702703e-14]))
(19, array([3.27690210e-14, 2.43178127e-14]))
(20, array([2.88633338e-14, 2.08752615e-14]

In [128]:
for md in [('decoder_null', 'n_do'), ('decoder_null', 'n_o'), ('full', 'n_do')]:
    print('')
    print(md)
    m = move_list[-3] #movement condition
    print(m)
    move_lqr = lqr_dm[md[0]]['move_lqr_n'][m[0], m[1], md[1]]
    
    num_seg = len(move_lqr['K_list'])
    
    for seg in range(num_seg):
        print('seg', seg)
        K_list = move_lqr['K_list'][seg]
        num_samples = K_list.shape[1]-1
        print(num_samples)
        for i in [36,37]:#range(num_samples):
            #print(i)
            K = K_list[0,i]
            u,s,vh = np.linalg.svd(K)
            u_dyn = u[:,:2]
            angles = scipy.linalg.subspace_angles(u_dyn, u_d)

            print(i, np.rad2deg(angles))
            print(s)


('decoder_null', 'n_do')
(7, 0)
('seg', 0)
38
(36, array([3.72945689e-12, 2.33624015e-12]))
[1.02280566e+00 7.53629204e-01 1.44277789e-14 9.67197377e-15
 6.28600808e-15 5.65154617e-15 4.02305174e-15 3.53910305e-15
 2.51117695e-15 1.35923034e-15 1.13744074e-15 9.68434913e-16
 7.66791402e-16 4.35363556e-16 2.32394049e-16 1.18815124e-16
 1.06711793e-16 5.64555508e-17 1.54388865e-17 1.12765319e-17]
(37, array([86.8357311 , 79.96875059]))
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0.]

('decoder_null', 'n_o')
(7, 0)
('seg', 0)
38
(36, array([3.73035477e-12, 2.33962889e-12]))
[1.02280566e+00 7.53629204e-01 1.14667523e-14 7.86521276e-17
 7.21488249e-17 4.01663656e-17 3.38534314e-17 2.70292921e-17
 2.46706193e-17 1.56652104e-17 1.29864676e-17 8.08922712e-18
 7.52995178e-18 6.23878566e-18 5.37544157e-18 4.44498010e-18
 2.99411744e-18 1.15847156e-18 8.18986681e-19 2.26928731e-19]
(37, array([86.8357311 , 79.96875059]))
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.

In [129]:
K.shape

(20L, 25L)

In [ ]:
#choose a trial of neural activity and confirm null variance